In [1]:
from vgg19 import BallerVgg
import os
import tensorflow as tf

import numpy as np
import time
import inspect
from PIL import Image
#import utils

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def iterate_minibatches(X_train, y_train, batchsize):
    while True:
        for i in range(0, X_train.shape[0] // batchsize):
            minibatch = X_train[i*batchsize:(i+1)*batchsize, :, :]
            y_minibatch = y_train[i*batchsize:(i+1)*batchsize]
            yield minibatch, y_minibatch

In [3]:
def one_hot_labels(y_train):
    averaged = np.sum(y_train, axis=1, keepdims=True) / 2
    y_train_buckets = np.zeros((averaged.shape[0], 1, 20))

    for i in range(averaged.shape[0]):
        val = (averaged / 1.5)[i, 0]
        y_train_buckets[i, 0, int(val)] = 1
    
    return y_train_buckets

In [4]:
y_train = np.load("./train/y_train.npy")
X_train = np.load("./train/X_train.npy", mmap_mode='r')

In [ ]:
def train(X_train, y_train):

    #batch1 = img1.reshape((1, 224, 224, 3))
    #batch2 = img2.reshape((1, 224, 224, 3))

    # with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=0.7)))) as sess:
    with tf.device('/cpu:0'):
        with tf.Graph().as_default():
            with tf.Session() as sess:
                vgg19_npy_path = '../../video2speed_data/vgg19.npy'
                vgg = BallerVgg(vgg19_npy_path)

                #images = tf.placeholder("float", (816, 224, 224, 3))
                images = tf.placeholder("float", (24, 224, 224, 3))
                labels_placeholder = tf.placeholder("int32", (12))
                
                with tf.name_scope("content_vgg"):
                    vgg.build(images)

                
                optimizer = tf.train.AdamOptimizer(.1)
                global_step = tf.Variable(0, name='global_step', trainable=False)
                cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_placeholder, logits=vgg.prob, name='xentropy')

                loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
                train_op = optimizer.minimize(loss, global_step=global_step)

                sess.run(tf.global_variables_initializer())

                #for x_batch, y_batch in iterate_minibatches(X_train, y_train, 408):
                #    x_batch = x_batch.reshape((816, 224, 224, 3))
                for x_batch, y_batch in iterate_minibatches(X_train, y_train, 12):
                    x_batch = x_batch.reshape((24, 224, 224, 3))
                    averaged = (np.sum(y_batch, axis=1) / 2) // 1.5
                    labels = averaged.astype(int)
          
                    feed_dict = {images: x_batch, labels_placeholder: labels}
                    _, cost = sess.run([train_op, loss], feed_dict = feed_dict)
                    print(cost)
                    vgg.save_npy(sess, npy_path="./vgg19-save.npy")

In [ ]:
train(X_train, y_train)

npy file loaded
build model started
2.99573
('file saved', './vgg19-save.npy')
